## Estimate crop area based on crop mask (single year)

**Author**: Hannah Kerner (hkerner@umd.edu)

**Description**: This notebook contains:
1. Code for computing the confusion matrix between the labeled reference sample and the crop mask
2. Calculations for the crop and noncrop area and accuracy estimates based on [Olofsson et al., 2014](https://www.sciencedirect.com/science/article/abs/pii/S0034425714000704)

To be added in the future:
- Code for thresholding the crop mask to a binary mask of 0 (noncrop) or 1 (crop)
- Code for clipping the rectangular crop mask to the bounds of a regional shapefile
- Code for creating a random stratified sample from the crop mask for labeling in CEO
- Code for sub-regional estimates (subsetting the reference sample according to admin2 bounds, e.g.), probably as a separate notebook

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
################################################################
### Set the custom paths that will be used for your analysis ###
### This should be the only code you need to change          ###
################################################################
mask_path = '/gpfs/data1/cmongp1/barker/binary_masks/reprojected/epsg32652_HLJ_2019.tif'
ceo_set1_path = '/gpfs/data1/cmongp1/barker/labeled_ceo/ceo-HLJ-2019-(Set-1)---v3-sample-data-2022-01-21.csv'
ceo_set2_path = '/gpfs/data1/cmongp1/barker/labeled_ceo/ceo-HLJ-2019-(Set-2)---v3-sample-data-2022-01-21.csv'

## 1. Load the crop mask

In [3]:
with rio.open(mask_path) as src:
    if src.meta['crs'] == 'epsg:4326':
        print('''WARNING: The map CRS is EPSG:4326. This means the map unit is degrees 
              and the pixel-wise areas will not be in meters. You need to reproject the map
              to the projection defined for the map's primary UTM zone (e.g., EPSG:32652).''')
    if src.meta['dtype'] != 'uint8':
        print('''WARNING: The map data type is %s but should be uint8. Make sure the map has
              been thresholded to convert to a binary mask of 0 (noncrop) or 1 (crop).''')
    else:
        print('Map CRS is %s. Loading map into memory.' % src.crs)
        crop_map = src.read(1).astype(np.uint8)

Map CRS is EPSG:32652. Loading map into memory.


In [4]:
# Plot the map to make sure it looks as expected
# This may take a while depending on the size of the map,
# so you may choose not to run this every time.
# plt.imshow(crop_map, cmap='YlGn');
# plt.axis('off');

## 2. Calculate the mapped area for each class

In [5]:
pixel_size = src.transform[0]
print('The pixel size is %f meters.' % pixel_size)

The pixel size is 7.978543 meters.


In [6]:
# Function to calculate mapped area in pixels or ha
def mapped_area(pred_map, unit='pixels', px_size=10):
    crop_px = np.where(pred_map.flatten() == 1)
    noncrop_px = np.where(pred_map.flatten() == 0)
    if unit == 'ha':
        # Multiply pixels by area per pixel and convert m to hectares
        crop_area = crop_px[0].shape[0] * (px_size*px_size) / 100000
        noncrop_area = noncrop_px[0].shape[0] * (px_size*px_size) / 100000
    elif unit == 'pixels':
        crop_area = int(crop_px[0].shape[0])
        noncrop_area = int(noncrop_px[0].shape[0])
    return crop_area, noncrop_area

In [7]:
crop_area_px, noncrop_area_px = mapped_area(crop_map)

print('Crop area [pixels] = %d' % crop_area_px)
print('Non-crop area [pixels] = %d' % noncrop_area_px)

Crop area [pixels] = 2507268291
Non-crop area [pixels] = 4538501847


In [8]:
tot_area_px = crop_area_px + noncrop_area_px
print('Total area [pixels] = %d' % tot_area_px)

Total area [pixels] = 7045770138


In [9]:
crop_area_ha, noncrop_area_ha = mapped_area(crop_map, unit='ha', px_size=pixel_size)

print('Crop area [ha] = %d' % crop_area_ha)
print('Non-crop area [ha] = %d' % noncrop_area_ha)

Crop area [ha] = 1596055
Non-crop area [ha] = 2889081


## 3. Load the labeled reference samples

There should be two sets of labels for the reference sample. We compare the labels from each set to filter out labels for which the labelers did not agree and thus we can't be confident about the true label.

In [10]:
ceo_set1 = pd.read_csv(ceo_set1_path)
ceo_set1.head()

,plotid,sampleid,lon,lat,email,flagged,collection_time,analysis_duration,imagery_title,imagery_attributions,sample_geom,Does this point lie on active cropland?
0,0,0,123.983561,47.216622,kmobley@umd.edu,False,2021-12-08 13:45,33.8 secs,Mapbox Satellite,NaN,POINT(123.9835605881125 47.216621572308455),Non-crop
1,1,1,125.086502,47.613475,kmobley@umd.edu,False,2021-12-06 18:26,18.0 secs,Planet Monthly Mosaics,NaN,POINT(125.0865015663171 47.61347456758538),Crop
2,2,2,131.693204,45.500411,kmobley@umd.edu,False,2021-12-06 18:27,11.0 secs,Planet Monthly Mosaics,NaN,POINT(131.69320402303472 45.50041127853154),Crop
3,3,3,132.594279,47.480545,kmobley@umd.edu,False,2021-12-06 18:27,16.0 secs,Planet Monthly Mosaics,NaN,POINT(132.594279295613 47.480545329246645),Crop
4,4,4,132.808439,46.979379,kmobley@umd.edu,False,2021-12-06 18:27,29.2 secs,Planet Monthly Mosaics,NaN,POINT(132.80843870683879 46.97937903978507),Crop


In [11]:
ceo_set2 = pd.read_csv(ceo_set2_path)
ceo_set2.head()

,plotid,sampleid,lon,lat,email,flagged,collection_time,analysis_duration,imagery_title,imagery_attributions,sample_geom,Does this point lie on active cropland?
0,0,0,123.983561,47.216622,kmobley@umd.edu,False,2021-12-08 13:54,23.4 secs,Mapbox Satellite,NaN,POINT(123.9835605881125 47.216621572308455),Non-crop
1,1,1,125.086502,47.613475,kmobley@umd.edu,False,2021-12-08 13:54,7.4 secs,Planet Monthly Mosaics,NaN,POINT(125.0865015663171 47.61347456758538),Crop
2,2,2,131.693204,45.500411,kmobley@umd.edu,False,2021-12-08 13:54,4.4 secs,Planet Monthly Mosaics,NaN,POINT(131.69320402303472 45.50041127853154),Crop
3,3,3,132.594279,47.480545,kmobley@umd.edu,False,2021-12-08 13:54,7.1 secs,Planet Monthly Mosaics,NaN,POINT(132.594279295613 47.480545329246645),Crop
4,4,4,132.808439,46.979379,kmobley@umd.edu,False,2021-12-08 13:54,4.8 secs,Planet Monthly Mosaics,NaN,POINT(132.80843870683879 46.97937903978507),Crop


In [ ]:
# Sometimes there are slight variations in the labeling question used. We store it here to avoid manual updates.
label_question = ceo_set1.columns[-1]

In [12]:
# Check for any NaNs / missing answers
ceo_set1[ceo_set1[label_question].isna()]

NameError: name 'label_question' is not defined

In [ ]:
# Check for any NaNs / missing answers
ceo_set2[ceo_set2[label_question].isna()]

In [ ]:
if ceo_set1.shape != ceo_set2.shape:
    print('ERROR: The size of the two dataframes does not match. Most likely, there is a duplicate in the plotid column resulting from an error in CEO. You need to delete the duplicate manually before continuing.')
    print(ceo_set1[ceo_set1.duplicated(subset=['plotid'])])
    print(ceo_set2[ceo_set2.duplicated(subset=['plotid'])])

In [ ]:
# Make sure the question and thus column name is correct for the project you are working on
ceo_agree = ceo_set1[ceo_set1[label_question] == 
                         ceo_set2[label_question]]

print('Number of samples that are in agreement: %d out of %d (%.2f%%)' % 
          (ceo_agree.shape[0], ceo_set1.shape[0], ceo_agree.shape[0]/ceo_set1.shape[0]*100))

In [ ]:
# Convert the pandas dataframe to a geodataframe
ceo_agree_geom = gpd.GeoDataFrame(ceo_agree, geometry=gpd.points_from_xy(ceo_agree.lon, ceo_agree.lat), crs='EPSG:4326')

In [ ]:
# The labeling platform CEO requires points to be in EPSG:4326. 
# Reproject to the same crs as the map.
ceo_agree_geom = ceo_agree_geom.to_crs(src.crs)

In [ ]:
# Plot them to make sure they look as expected
ceo_agree_geom.plot();

## 4. Get the mapped class for each of the reference samples

In [ ]:
for r, row in ceo_agree_geom.iterrows():
    # transform lon, lat to pixel coordinates
    lon, lat = row['geometry'].y, row['geometry'].x
    px, py = src.index(lat, lon)
    ceo_agree_geom.loc[r,'Mapped class'] = crop_map[px, py]

In [ ]:
ceo_agree_geom.head()

In [ ]:
# Make sure none of them are nodata
ceo_agree[ceo_agree_geom['Mapped class'] == 3]

## 5. Compute the confusion matrix between the mapped classes and reference labels

In [ ]:
# Convert the CEO string label to an integer label
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Crop', 'Reference label'] = 1
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Non-crop', 'Reference label'] = 0
# Account for alternate spellings that may be pressent
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'Noncrop', 'Reference label'] = 0
ceo_agree_geom.loc[ceo_agree_geom[label_question] == 'NonCrop', 'Reference label'] = 0

ceo_agree_geom['Reference label'] = ceo_agree_geom['Reference label'].astype(np.uint8)
ceo_agree_geom.head()

In [ ]:
# Compute confusion matrix
y_true = np.array(ceo_agree_geom['Reference label']).astype(np.uint8)
y_pred = np.array(ceo_agree_geom['Mapped class']).astype(np.uint8)
confusion_matrix(y_true, y_pred)

In [ ]:
# Extract and print confusion matrix values with element descriptions
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('True negatives: %d' % tn)
print('False positives: %d' % fp)
print('False negatives: %d' % fn)
print('True positives: %d' % tp)

## 6. Adjust mapped area using confusion matrix to compute area estimates

$W_h$ is the proportion of mapped area for each class 

In [ ]:
wh_crop = crop_area_px / tot_area_px
print('Wh_crop = %f' % wh_crop)

wh_noncrop = noncrop_area_px / tot_area_px
print('Wh_noncrop = %f' % wh_noncrop)

Compute the fraction of the proportional area of each class that was mapped as each category in the confusion matrix

In [ ]:
tp_area = tp / (tp + fp) * wh_crop
fp_area = fp / (tp + fp) * wh_crop
fn_area = fn / (fn + tn) * wh_noncrop
tn_area = tn / (fn + tn) * wh_noncrop

print('%f \t %f \n %f \t %f' % (tp_area, fp_area, fn_area, tn_area))

$U_i$ is the user's accuracy (i.e., precision) for each mapped class. We calculate it here in terms of proportion of area computed in the last cell.

In [ ]:
u_crop = tp_area / (tp_area + fp_area)
print('U_crop = %f' % u_crop)

u_noncrop = tn_area / (tn_area + fn_area)
print('U_noncrop = %f' % u_noncrop)

$V(U_i)$ is the estimated variance of user accuracy for each mapped class.

In [ ]:
v_u_crop = u_crop * (1-u_crop) / (tp + fp)
print('V(U)_crop = %f' % v_u_crop)

v_u_noncrop = u_noncrop * (1-u_noncrop) / (fn + tn)
print('V(U)_noncrop = %f' % v_u_noncrop)

$S(U_i)$ is the estimated standard error of user accuracy for each mapped class.

In [ ]:
s_u_crop = np.sqrt(v_u_crop)
print('S(U)_crop = %f' % s_u_crop)

s_u_noncrop = np.sqrt(v_u_noncrop)
print('S(U)_noncrop = %f' % s_u_noncrop)

Get the 95% confidence interval for User's accuracy

In [ ]:
u_crop_err = s_u_crop * 1.96
print('95%% CI of User accuracy for crop = %f' % u_crop_err)

u_noncrop_err = s_u_noncrop * 1.96
print('95%% CI of User accuracy for noncrop = %f' % u_noncrop_err)

$P$ is the producer's accuracy (i.e., recall). We calculate it here in terms of proportion of area.

In [ ]:
p_crop = tp_area / (tp_area + fn_area)
print('P_crop = %f' % p_crop)

p_noncrop = tn_area / (tn_area + fp_area)
print('P_noncrop = %f' % p_noncrop)

$N_j$ is the estimated marginal total number of pixels of each reference class $j$

In [ ]:
n_j_crop = (crop_area_px * tp) / (tp + fp) + (noncrop_area_px * fn) / (fn + tn)
print('N_j_crop = %f' % n_j_crop)

n_j_noncrop = (crop_area_px * fp) / (tp + fp) + (noncrop_area_px * tn) / (fn + tn)
print('N_j_crop = %f' % n_j_noncrop)

In [ ]:
expr1_crop = crop_area_px**2 * (1-p_crop)**2 * u_crop * (1-u_crop) / (tp + fp - 1)
print('expr1 crop = %f' % expr1_crop)

expr1_noncrop = noncrop_area_px**2 * (1-p_noncrop)**2 * u_noncrop * (1-u_noncrop) / (fp + tn - 1)
print('expr1 noncrop = %f' % expr1_noncrop)

In [ ]:
# Warning: depending on the size of your map, you may get an overflow warning here, e.g.
# RuntimeWarning: overflow encountered in long_scalars
# Need to figure out if we can correct this...
expr2_crop = p_crop**2 * (noncrop_area_px**2 * fn / (fn + tn) * (1 - fn / (fn + tn)) / (fn + tn - 1))
print('expr2 crop = %f' % expr2_crop)

expr2_noncrop = p_crop**2 * (crop_area_px**2 * fp / (fp + tp) * (1 - fp / (fp + tp)) / (fp + tp - 1))
print('expr2 noncrop = %f' % expr2_noncrop)

$V(P_i)$ is the estimated variance of producer's accuracy for each mapped class.

In [ ]:
v_p_crop = (1 / n_j_crop**2) * (expr1_crop + expr2_crop)
print('V(P) crop = %f' % v_p_crop)

v_p_noncrop = (1 / n_j_noncrop**2) * (expr1_noncrop + expr2_noncrop)
print('V(P) noncrop = %f' % v_p_noncrop)

$S(P_i)$ is the estimated standard error of producer accuracy for each mapped class.

In [ ]:
# Warning: depending on the size of your map, you may get an overflow warning here, e.g.
# RuntimeWarning: overflow encountered in long_scalars
# Need to figure out if we can correct this...
s_p_crop = np.sqrt(v_p_crop)
print('S(P) crop = %f' % s_p_crop)

s_p_noncrop = np.sqrt(v_p_noncrop)
print('S(P) noncrop = %f' % s_p_noncrop)

Get the 95% confidence interval for Producer's accuracy

In [ ]:
p_crop_err = s_p_crop * 1.96
print('95%% CI of Producer accuracy for crop = %f' % p_crop_err)

p_noncrop_err = s_p_noncrop * 1.96
print('95%% CI of Producer accuracy for noncrop = %f' % p_noncrop_err)

$O$ is the overall accuracy. We calculate it here in terms of proportion of area.

In [ ]:
acc = tp_area + tn_area
print('Overall accuracy = %f' % acc)

$V(O)$ is the estimated variance of the overall accuracy

In [ ]:
v_acc = wh_crop**2 * u_crop * (1-u_crop) / (tp + fp - 1) + \
        wh_noncrop**2 * u_noncrop * (1-u_noncrop) / (fn + tn - 1)
print('V(O) = %f' % v_acc)

$S(O)$ is the estimated standard error of the overall accuracy

In [ ]:
s_acc = np.sqrt(v_acc)
print('S(O) = %f' % s_acc)

Get the 95% confidence interval for overall accuracy

In [ ]:
acc_err = s_acc * 1.96
print('95%% CI of overall accuracy = %f' % acc_err)

$A_{pixels}$ is the adjusted map area in units of pixels

In [ ]:
a_pixels_crop = tot_area_px * (tp_area + fn_area)
print('A^[pixels] crop = %f' % a_pixels_crop)

a_pixels_noncrop = tot_area_px * (tn_area + fp_area)
print('A^[pixels] noncrop = %f' % a_pixels_noncrop)

$A_{ha}$ is the adjusted map area in units of hectares

In [ ]:
a_ha_crop = a_pixels_crop * (pixel_size*pixel_size) / (100*100)
print('A^[ha] crop = %f' % a_ha_crop)

a_ha_noncrop = a_pixels_noncrop * (pixel_size*pixel_size) / (100*100)
print('A^[ha] noncrop = %f' % a_ha_noncrop)

The following equations are used to estimate the standard error for the area. They are based on the calculations in Olofsson et al., 2014.

In [ ]:
S_pk_crop = np.sqrt((wh_crop * tp_area - tp_area**2) / (tp + fp - 1) + \
                     (wh_noncrop * fn_area - fn_area**2) / (fn + tn - 1)) * tot_area_px
print('S_pk_crop = %f' % S_pk_crop)

S_pk_noncrop = np.sqrt((wh_crop * fp_area - fp_area**2) / (tp + fp - 1) + \
                        (wh_noncrop * tn_area - tn_area**2) / (fn + tn - 1)) * tot_area_px
print('S_pk_noncrop = %f' % S_pk_noncrop)

Multiply $S(p_k)$ by 1.96 to get the margin of error for the 95% confidence interval

In [ ]:
a_pixels_crop_err = S_pk_crop * 1.96
print('Crop area standard error 95%% confidence interval [pixels] = %f' % a_pixels_crop_err)

a_pixels_noncrop_err = S_pk_noncrop * 1.96
print('Non-crop area standard error 95%% confidence interval [pixels] = %f' % a_pixels_noncrop_err)

In [ ]:
a_ha_crop_err = a_pixels_crop_err * (pixel_size**2) / (100**2)
print('Crop area standard error 95%% confidence interval [ha] = %f' % a_ha_crop_err)

a_ha_noncrop_err = a_pixels_noncrop_err * (pixel_size**2) / (100**2)
print('Non-crop area standard error 95%% confidence interval [ha] = %f' % a_ha_noncrop_err)

Summary of the final estimates of accuracy and area with standard error at 95% confidence intervals:

In [ ]:
summary = pd.DataFrame([[a_ha_crop, a_ha_noncrop],
                        [a_ha_crop_err, a_ha_noncrop_err],
                        [u_crop, u_noncrop],
                        [u_crop_err, u_noncrop_err],
                        [p_crop, p_noncrop],
                        [p_crop_err, p_noncrop_err],
                        [acc, acc],
                        [acc_err, acc_err]
                       ],
                       index=pd.Index(['Estimated area [ha]', '95% CI of area [ha]', 'User accuracy',
                                       '95% CI of user acc', 'Producer accuracy', '95% CI of prod acc',
                                       'Overall accuracy', '95% CI of overall acc']),
                       columns=['Crop', 'Non-crop'])

summary.round(2)